In [1]:
# 导入
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
import joblib
from sklearn.preprocessing import StandardScaler

In [3]:
df_name = './data/附件2/'+ 'M202.csv'#'M201.csv'
df = pd.read_csv(df_name)

In [4]:
error_list=['物料推送装置故障1001',
'物料检测装置故障2001',
'填装装置检测故障4001',
'填装装置定位故障4002',
'填装装置填装故障4003',
'加盖装置定位故障5001',
'加盖装置加盖故障5002',
'拧盖装置定位故障6001',
'拧盖装置拧盖故障6002'
]

In [5]:
n = len(df.columns)
# 选择倒数前九列
dfx = df.iloc[:, :n-9]
# 删除生产线编号列
dfx = dfx.drop(['生产线编号'], axis=1)
X=dfx
# 特征缩放
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
def wdata(df_error,i):
    # 创建DataFrame
    df = pd.DataFrame(df_error, columns=["日期", "时间"])

    # 计算连续时间之间的差值
    df["TimeDiff"] = df["时间"].diff()

    # 创建一个掩码，用于标识新的连续时间段的开始
    mask = df["TimeDiff"] != 1

    # 分组连续时间段并计算起始时间和持续时间
    result = df.groupby(["日期", mask.cumsum()])["时间"].agg(["first", "count"]).reset_index()
    # 重命名列
    result.columns = ["日期", "序号", "开始时间", "持续时间"]
    result = result.drop(['序号'], axis=1)
    result_name = 'M202_pred' +error_list[i] +'.xlsx'
    result.to_excel(result_name)
    

In [7]:
for i in range(len(error_list)):#len(error_list)
    name = error_list[i] + '.pkl'
    # 加载之前保存的集成模型
    loaded_ensemble = joblib.load(name)
    # 使用已加载的集成模型进行预测
    predictions = loaded_ensemble.predict(X)
    dfx[error_list[i]]= predictions
    print(dfx[error_list[i]].value_counts())
    df_error = dfx[dfx[error_list[i]]==1]
    wdata(df_error,i)
    print(f'{error_list[i]}  finish')


物料推送装置故障1001
0    7522843
1       3876
Name: count, dtype: int64
物料推送装置故障1001  finish
物料检测装置故障2001
0    7480770
1      45949
Name: count, dtype: int64
物料检测装置故障2001  finish
填装装置检测故障4001
0    7231479
1     295240
Name: count, dtype: int64
填装装置检测故障4001  finish
填装装置定位故障4002
0    7522083
1       4636
Name: count, dtype: int64
填装装置定位故障4002  finish
填装装置填装故障4003
0    7463632
1      63087
Name: count, dtype: int64
填装装置填装故障4003  finish
加盖装置定位故障5001
0    7473204
1      53515
Name: count, dtype: int64
加盖装置定位故障5001  finish
加盖装置加盖故障5002
0    7475945
1      50774
Name: count, dtype: int64
加盖装置加盖故障5002  finish
拧盖装置定位故障6001
0    7448301
1      78418
Name: count, dtype: int64
拧盖装置定位故障6001  finish
拧盖装置拧盖故障6002
0    7456840
1      69879
Name: count, dtype: int64
拧盖装置拧盖故障6002  finish
